In [10]:
import requests
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import clear_output
import time
from datetime import datetime
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Font, Alignment
from openpyxl.utils import get_column_letter


In [3]:
def fetch_crypto_data():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        'vs_currency': 'usd',
        'order': 'market_cap_desc',
        'per_page': 50,
        'page': 1,
        'sparkline': False
    }
    
    response = requests.get(url, params=params)
    data = response.json()
    
    df = pd.DataFrame(data)[['name', 'symbol', 'current_price', 'market_cap', 
                           'total_volume', 'price_change_percentage_24h',
                           'high_24h', 'low_24h']]
    
    df.columns = ['Name', 'Symbol', 'Price_USD', 'Market_Cap', 'Volume_24h',
                 'Price_Change_24h', 'High_24h', 'Low_24h']
    return df

In [4]:
def analyze_crypto_data(df):
    
    total_market_cap = df['Market_Cap'].sum()
    top_5_market_cap = df.nlargest(5, 'Market_Cap')[['Name', 'Market_Cap', 'Price_USD']]
    
    best_performers = df.nlargest(3, 'Price_Change_24h')[['Name', 'Price_Change_24h']]
    worst_performers = df.nsmallest(3, 'Price_Change_24h')[['Name', 'Price_Change_24h']]
    
    high_volume = df.nlargest(5, 'Volume_24h')[['Name', 'Volume_24h']]
    
    df['Volatility'] = ((df['High_24h'] - df['Low_24h']) / df['Low_24h'] * 100)
    most_volatile = df.nlargest(5, 'Volatility')[['Name', 'Volatility']]
    
    return {
        'total_market_cap': total_market_cap,
        'top_5_market_cap': top_5_market_cap,
        'best_performers': best_performers,
        'worst_performers': worst_performers,
        'high_volume': high_volume,
        'most_volatile': most_volatile
    }

In [5]:
def create_visualizations(df):
    """Create interactive visualizations of the crypto data"""
    
    market_cap_fig = px.treemap(df, 
                               path=['Name'],
                               values='Market_Cap',
                               title='Market Cap Distribution')
    
    price_change_fig = px.bar(df.nlargest(10, 'Market_Cap'),
                             x='Name',
                             y='Price_Change_24h',
                             title='Price Change % (24h) for Top 10 Cryptocurrencies')
    
    volume_market_fig = px.scatter(df,
                                  x='Market_Cap',
                                  y='Volume_24h',
                                  text='Name',
                                  title='Volume vs Market Cap',
                                  log_x=True,
                                  log_y=True)
    
    return market_cap_fig, price_change_fig, volume_market_fig

In [6]:
def create_market_cap_treemap(df):
    
    total_market_cap = df['Market_Cap'].sum()
    df['Market_Cap_Percentage'] = (df['Market_Cap'] / total_market_cap * 100).round(2)
    
    df['Market_Cap_Display'] = df['Market_Cap'].apply(lambda x: f"${x:,.0f}")
    
    df['CustomText'] = df.apply(lambda row: (
        f"Name: {row['Name']}<br>" +
        f"Market Cap: {row['Market_Cap_Display']}<br>" +
        f"Market Share: {row['Market_Cap_Percentage']}%<br>" +
        f"Price: ${row['Price_USD']:,.2f}<br>" +
        f"24h Change: {row['Price_Change_24h']:.2f}%"
    ), axis=1)
    
    fig = px.treemap(
        df,
        path=['Name'],
        values='Market_Cap',
        custom_data=['CustomText'], 
        color='Price_Change_24h',   
        color_continuous_scale='RdYlGn',  
        title='Cryptocurrency Market Cap Distribution'
    )
    
    fig.update_traces(
        textinfo="label+value",
        hovertemplate="%{customdata[0]}<extra></extra>", 
        textfont=dict(size=16),
    )
    
    fig.update_layout(
        uniformtext=dict(minsize=10),
        coloraxis_colorbar=dict(
            title="24h Price Change %"
        ),
        margin=dict(t=50, l=25, r=25, b=25)
    )
    
    return fig

In [7]:
def live_crypto_analysis():
    """Perform live analysis with 5-minute updates"""
    while True:
        clear_output(wait=True)
        
        df = fetch_crypto_data()
        analysis = analyze_crypto_data(df)
        
        print(f"Last Updated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
        print("\n=== Market Overview ===")
        print(f"Total Market Cap: ${analysis['total_market_cap']:,.2f}")
        
        print("\n=== Top 5 by Market Cap ===")
        print(analysis['top_5_market_cap'].to_string())
        
        print("\n=== Best Performers (24h) ===")
        print(analysis['best_performers'].to_string())
        
        print("\n=== Worst Performers (24h) ===")
        print(analysis['worst_performers'].to_string())
        
        print("\n=== Most Volatile (24h) ===")
        print(analysis['most_volatile'].to_string())
        
        market_cap_fig, price_change_fig, volume_market_fig = create_visualizations(df)
        market_cap_fig.show()
        price_change_fig.show()
        volume_market_fig.show()
        treemap_fig = create_market_cap_treemap(df)
        treemap_fig.show()
        
        time.sleep(300)

I WILL RUN THIS BELOW LIVE CRYPTO ANALYSIS AT END AFTER EXCECUTING THE EXCEL CELLS BECAUSE IT IS EVERUNNING SCRIPT AS IT NEEDS TO BE UPDATED EVERY 5 MIN

In [ ]:
# live_crypto_analysis()

NOW UNDER IS THE CODE FOR CREATING EXCEL FILE WITH ALL THE KEY POINT COLUMNS WHICH CHANGE EVERY 5 MINUTES.

In [11]:
def format_excel_sheet(ws):

    header_fill = PatternFill(start_color="1F4E78", end_color="1F4E78", fill_type="solid")
    header_font = Font(color="FFFFFF", bold=True)
    
    for cell in ws[1]:
        cell.fill = header_fill
        cell.font = header_font
        cell.alignment = Alignment(horizontal='center')
    
    for column in ws.columns:
        max_length = 0
        column_letter = get_column_letter(column[0].column)
        
        for cell in column:
            try:
                max_length = max(max_length, len(str(cell.value)))
            except:
                pass
            
        adjusted_width = max_length + 2
        ws.column_dimensions[column_letter].width = adjusted_width

In [12]:
def update_excel_report(df, analysis):
    """Create/update Excel report with latest data"""
    file_name = 'crypto_live_data.xlsx'
    
    with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name='Live Data', index=False)
        
        analysis_data = pd.DataFrame([
            {'Metric': 'Total Market Cap', 
             'Value': f"${analysis['total_market_cap']:,.2f}"},
            {'Metric': 'Top Cryptocurrency', 
             'Value': analysis['top_5_market_cap'].iloc[0]['Name']},
            {'Metric': 'Best Performer (24h)', 
             'Value': f"{analysis['best_performers'].iloc[0]['Name']} ({analysis['best_performers'].iloc[0]['Price_Change_24h']:.2f}%)"},
            {'Metric': 'Worst Performer (24h)', 
             'Value': f"{analysis['worst_performers'].iloc[0]['Name']} ({analysis['worst_performers'].iloc[0]['Price_Change_24h']:.2f}%)"},
            {'Metric': 'Last Updated', 
             'Value': datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
        ])
        analysis_data.to_excel(writer, sheet_name='Analysis', index=False)
        
        top_5_data = df.head()
        top_5_data.to_excel(writer, sheet_name='Top 5 Crypto', index=False)
        
        for sheet_name in writer.sheets:
            format_excel_sheet(writer.sheets[sheet_name])
    
    print(f"Excel report updated at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    return file_name

In [13]:
def live_crypto_analysis_with_excel():
    """Perform live analysis with Excel updates every 5 minutes"""
    while True:
        clear_output(wait=True)
        
        df = fetch_crypto_data()
        analysis = analyze_crypto_data(df)
        
        excel_file = update_excel_report(df, analysis)
        
        print(f"Last Updated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Excel report updated: {excel_file}")
        
        print("\n=== Market Overview ===")
        print(f"Total Market Cap: ${analysis['total_market_cap']:,.2f}")
        
        print("\n=== Top 5 by Market Cap ===")
        print(analysis['top_5_market_cap'].to_string())
        
        print("\n=== Best Performers (24h) ===")
        print(analysis['best_performers'].to_string())
        
        print("\n=== Worst Performers (24h) ===")
        print(analysis['worst_performers'].to_string())
        
        treemap_fig = create_market_cap_treemap(df)
        treemap_fig.show()
        
        time.sleep(300)

In [14]:
df = fetch_crypto_data()
analysis = analyze_crypto_data(df)
excel_file = update_excel_report(df, analysis)
print(f"Excel report created: {excel_file}")

Excel report updated at 2025-02-11 14:15:51
Excel report created: crypto_live_data.xlsx


In [ ]:
live_crypto_analysis()

Last Updated: 2025-02-11 14:36:27

=== Market Overview ===
Total Market Cap: $3,176,314,476,538.00

=== Top 5 by Market Cap ===
       Name     Market_Cap  Price_USD
0   Bitcoin  1946604393771   98194.00
1  Ethereum   327326690295    2715.77
2       XRP   144769678616       2.50
3    Tether   141897018030       1.00
4    Solana   100067219506     204.99

=== Best Performers (24h) ===
        Name  Price_Change_24h
8    Cardano          14.68936
17  Litecoin          12.02749
15       Sui          11.18719

=== Worst Performers (24h) ===
      Name  Price_Change_24h
30  MANTRA          -2.60531
44   sUSDS          -0.36209
22    USDS          -0.09330

=== Most Volatile (24h) ===
         Name  Volatility
8     Cardano   17.329995
40  Bittensor   13.444764
17   Litecoin   13.356933
15        Sui   11.949686
33       Pepe   10.432034
